# Collect This Year's Tournament Data
This should only be ran one time, and on the correct day - when all regular season games are finished

In [1]:
# our imports for the model
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import lxml.html as lh

In [2]:
import os

class collect_pipeline(object):

    def make_folder(self):
        if not os.path.exists('./tournament_data'):
            os.mkdir('./tournament_data')
            
        


In [3]:

class bball_scraper():
   

    def __init__(self ):
        self.start_urls = []
        self.data_one = None
    
    # url_list is a list of dictionary's. each has a 'name' id' , 'year' , 'url'
    # for the second group of urls - data will be none type
    def set_urls(self , url_list):
        self.start_urls = url_list
    
    # this is the first set of urls. only need the first url from one table
    def parse_one(self):
        season_stats = pd.DataFrame()   # start with an empty dataframe
        
        for item in self.start_urls:
            # extact the information we need
            team_id = item['id']
            team_name = item['name']
            url = item['url']
            year = item['year']
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "lxml")
            tables = soup.findAll('table')    # search for tables 
            # convert to a dataframe and label the data
            df = pd.read_html(str(tables[1]))[0]      # select the table of interest into a pandas dataframe  
            df.drop([1 , 2, 3] , inplace=True)
            df = df.replace('Team' , team_name)   # want the name of the team 
            df['Team_ID'] = team_id        # put in the team ID's
            df['Date'] = year  # only want the year in this column
            df = df.rename(columns={'Unnamed: 0': 'Team'})

            
            season_stats = season_stats.append(df , ignore_index = True , sort=False)
           # season_stats = pd.concat([season_stats, df] ,  axis=0) # add this to the season stats empty dataframe we started with
              # simply clean up the columns 
            
            # save the data
            self.data = season_stats  
            
        # this is the second set of urls - more than one row from the table
    def parse_two(self):
        wl_teams = pd.DataFrame()      # initialize and empty dataframe
        for item in self.start_urls:
                # extract the info
            team_idea = item['id']
            team_name = item['name']
            url = item['url']
                   
                # parse the page
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "lxml")
            tables = soup.findAll('table')
        
                # build a dataframe of the win loss data
            win_loss_df = pd.read_html(str(tables))[0]
            win_loss_6_df = win_loss_df.head(12)
            win_loss_6_df['team'] = team_name

            wl_teams = wl_teams.append(win_loss_6_df , ignore_index=True)    

        # clean the dataframe

        wl_teams.columns = wl_teams.columns.droplevel(level=0)
        wl_teams['team'] = wl_teams[""]
        wl_teams.drop(""  , axis=1)
        
        self.data = wl_teams